In [115]:
import pandas as pd
from datetime import datetime
import numpy as np
import glob
import os
import xlrd
import re
import openpyxl
import random

# custom file that maps state names to abbreviations
from abbreviation_conversion import abbrev_to_us_state

## Clean up prescription data


In [116]:
def find_year(TRANSACTION_DATE):
    """
    Args:
        TRANSACTION_DATE (str): date in format MMDDYYYY

    Returns:
        int: year
    """
    TRANSACTION_DATE = str(TRANSACTION_DATE)
    
    return int(TRANSACTION_DATE[-4:])

# quick test 
print(f"testing the find_year function: {find_year(12202013)}")


def find_month(TRANSACTION_DATE):
    """
    Args:
        TRANSACTION_DATE (str): date in format MMDDYYYY

    Returns:
        int: month
    """
    TRANSACTION_DATE = str(TRANSACTION_DATE)

    if len(TRANSACTION_DATE) == 8:
        return int(TRANSACTION_DATE[:2])
    else:
        return int(TRANSACTION_DATE[:1])
    

# quick test 
print(f"testing the find_month function: {find_month(12202013)}")

testing the find_year function: 2013
testing the find_month function: 12


In [3]:
# to load in the data, we need to truncate the amount of columns we use as well as the states
cols_to_keep = ["BUYER_STATE", "BUYER_ZIP", "BUYER_COUNTY", "DRUG_CODE", "DRUG_NAME", "QUANTITY", "TRANSACTION_DATE"]

# we know we need Florida, Texas, and Washington
states = ["FL", "TX", "WA"]
# since we are normalizing based on population, I think we should pick states that are regionally close to our target states
# we can change this later as a group, but I have these selected below:

# Florida comparison states
fl_states = ["PA", "MI", "NC"]

# Texas comparison states
tx_states = ["IL", "MA", "MI"]

# Washington comparison states
wa_states = ["NC", "CO", "MD"]

# create list of all states to use
variable_states = []
variable_states.extend(fl_states)
variable_states.extend(tx_states)
variable_states.extend(wa_states)

# append variable states to our original list
states.extend(variable_states)


# create separate list of only florida and washington states for prescription data
prescription_states = [state for state in states if state not in ["IL", "MA", "MI", "TX"]]

# NC is appearing twice as it's a comparison state for both target states
# making this a set will remove the duplicate
prescription_states = list(set(prescription_states))


### Loading in individual state prescription data

The Washington Post article associated with our data states that data from 2013 and 2014 was only recently added. Resultingly, we found that it was missing from the large dataset of all states. However, upon further digging, we found that these years were present **on an individual state level**, so we will load these in and concatenate them with our larger dataframe above.

Without chunking, the below takes 30 seconds for each file to load in. With chunking, this is reduced to about 4 seconds per record, so please make sure to leave this in its current format.



current issues:
- way more records for 2013 and 2014
- non WA states not being read in

In [11]:
# set FINAL cols we want
cols_to_keep = ["BUYER_STATE", "BUYER_ZIP", "BUYER_COUNTY", "DRUG_CODE", "DRUG_NAME", "TRANSACTION_DATE", "MME_Conversion_Factor", "dos_str", "DOSAGE_UNIT", "CALC_BASE_WT_IN_GM"]

# set additional columns we need to calculate MME
chunk_cols = ["dos_str", "DOSAGE_UNIT", "MME_Conversion_Factor",]



path = r'00_source_data/state_prescriptions' # point to correct folder
filenames = glob.glob(path + "/*.csv") # select all text files in folder

assert len(filenames) == 7, "There should be 7 files in the folder - check that we don't have a missing state"

df = pd.DataFrame() # empty df - will store data from all txt files




for f, count in zip(filenames, range(len(filenames))):

    it = pd.read_csv(f, chunksize=1_000_000, usecols = cols_to_keep) # may have to change chunksize depending on your computer's memory
    
    temp_df = pd.DataFrame()

    for chunk in it:

        # ensure dtypes for faster calculation below
        float_cols = ["MME_Conversion_Factor", "dos_str", "DOSAGE_UNIT", "CALC_BASE_WT_IN_GM"]
        chunk[float_cols] = chunk[float_cols].astype("float64")

        # calculate MME
        chunk["MME"] = chunk["dos_str"] * chunk["MME_Conversion_Factor"] * chunk["DOSAGE_UNIT"]

        chunk = chunk[cols_to_keep]
        # ensure we're working in the correct date range
        #filtered_chunk = chunk[chunk["year"] > 2002]
        #filtered_chunk = filtered_chunk[filtered_chunk["year"] < 2016]

        # extract year out of date column
        chunk["year"] = chunk["TRANSACTION_DATE"].apply(lambda x: int(find_year(x)))

        # calculate int cols
        
        int_cols = ["BUYER_ZIP", "year"]
        chunk[int_cols] = chunk[int_cols].astype("int64")

        temp_df = pd.concat([temp_df, chunk])

    print(f"Finished processing record {count} of filename {f}")
    df = pd.concat([df_prescriptions, temp_df], axis=0, ignore_index=True)

df.drop(columns={"TRANSACTION_DATE"}, inplace=True)

Finished processing record 0 of filename 00_source_data/state_prescriptions\arcos-co-statewide-itemized.csv
Finished processing record 1 of filename 00_source_data/state_prescriptions\arcos-fl-statewide-itemized.csv
Finished processing record 2 of filename 00_source_data/state_prescriptions\arcos-md-statewide-itemized.csv
Finished processing record 3 of filename 00_source_data/state_prescriptions\arcos-mi-statewide-itemized.csv
Finished processing record 4 of filename 00_source_data/state_prescriptions\arcos-nc-statewide-itemized.csv
Finished processing record 5 of filename 00_source_data/state_prescriptions\arcos-pa-statewide-itemized.csv
Finished processing record 6 of filename 00_source_data/state_prescriptions\arcos-wa-statewide-itemized.csv


In [12]:
df["MME"] = df["CALC_BASE_WT_IN_GM"] * df["MME_Conversion_Factor"]

assert len(df[df["MME"].isnull()]) == 0, "There should be no missing MME calculations"

In [13]:
# check for null counties & no shipments in counties


In [14]:
# check for null counties & no shipments in counties

# check for null counties
#df_prescriptions[df_prescriptions["BUYER_COUNTY"].isnull()]
# uncommenting the above line shows us only one missing county - zip code 34635
# this is bellair beach, so we'll fill this value in and add an assert to ensure no more missing counties

# replace buyer county where buyer zip is equal to 34635 with bellair beach
df.loc[df["BUYER_ZIP"] == 34635, "BUYER_COUNTY"] = "Pinellas"

In [15]:
assert len(df[df["BUYER_COUNTY"].isnull()]) == 0, "There should be no missing counties"


### group by state, county, and year

Turns unit of observation into one MME calculation per county-year

In [16]:
df[df["BUYER_COUNTY"] == "PUEBLO"]

,BUYER_STATE,BUYER_ZIP,BUYER_COUNTY,DRUG_CODE,DRUG_NAME,MME_Conversion_Factor,dos_str,DOSAGE_UNIT,CALC_BASE_WT_IN_GM,year,MME
0,CO,81005,PUEBLO,9193,HYDROCODONE,1.0,5.0,150.0,0.454050,2009,0.454050
324,CO,81004,PUEBLO,9193,HYDROCODONE,1.0,7.5,500.0,2.270250,2006,2.270250
325,CO,81004,PUEBLO,9193,HYDROCODONE,1.0,10.0,100.0,0.605400,2006,0.605400
326,CO,81004,PUEBLO,9193,HYDROCODONE,1.0,5.0,500.0,1.513500,2006,1.513500
327,CO,81004,PUEBLO,9143,OXYCODONE,1.5,7.5,100.0,0.672375,2006,1.008563
...,...,...,...,...,...,...,...,...,...,...,...
3818207,CO,81005,PUEBLO,9193,HYDROCODONE,1.0,5.0,300.0,0.908100,2013,0.908100
3818208,CO,81005,PUEBLO,9193,HYDROCODONE,1.0,5.0,1000.0,3.027000,2014,3.027000
3818209,CO,81005,PUEBLO,9193,HYDROCODONE,1.0,7.5,500.0,2.270250,2014,2.270250
3818210,CO,81005,PUEBLO,9193,HYDROCODONE,1.0,5.0,500.0,1.513500,2014,1.513500


In [19]:
# groupby to get one row per county year
# keeping zip for now, may not be necessary

df_prescriptions = df.groupby(["BUYER_STATE", "BUYER_COUNTY", "year"])["MME"].sum().reset_index()


#df_prescriptions = df_grouped.copy() # temporary - will remove this later

In [20]:
df_prescriptions[df_prescriptions["BUYER_COUNTY"] == "PUEBLO"].head(50)

,BUYER_STATE,BUYER_COUNTY,year,MME
423,CO,PUEBLO,2006,66467.019279
424,CO,PUEBLO,2007,80805.048150
425,CO,PUEBLO,2008,98193.220958
426,CO,PUEBLO,2009,115942.791973
427,CO,PUEBLO,2010,141464.937486
428,CO,PUEBLO,2011,180468.041823
429,CO,PUEBLO,2012,204180.305891
430,CO,PUEBLO,2013,146468.052559
431,CO,PUEBLO,2014,121860.909022


problems
- read huge arcos file, never unzipped (read as compressed zip file)
? limited cols taking in, filtered for specific states he wanted
 - couple of transofmrs - pulled year col, std morpine calculation
? saved memory! -> last thing for each chunk, add state abbrev for county name
then grouped chunk, chunk size fairly small (maybe he means agg to year now?)
    go through x chunks, groupbys here he means
    getting to pt where more manageable on memory + easier for other team members


other thing:
if you leave zipped, read chunks and do transforms on chunks, it'll get done in 12min (about)


**NOT CURRENTLY USING THE BELOW CELL**

In [50]:
"""
# set FINAL cols we want
cols_to_keep = ["BUYER_STATE", "BUYER_ZIP", "BUYER_COUNTY", "DRUG_CODE", "DRUG_NAME", "TRANSACTION_DATE", "MME_Conversion_Factor", "dos_str", "DOSAGE_UNIT"]

# set additional columns we need to calculate MME
chunk_cols = ["dos_str", "DOSAGE_UNIT", "MME_Conversion_Factor"]


df_prescriptions = pd.DataFrame() # empty df - will store data from all txt files

# need 1 row per county month**** could aggergate it here also?
it = pd.read_csv("00_source_data/arcos_all_washpost.tsv.gz", chunksize=1_000_000, sep='\t', usecols=cols_to_keep, low_memory=False) # may have to change chunksize depending on your computer's memory
cols_to_keep.extend(["year", "MME"])

temp_df = pd.DataFrame()
counter = 0
for chunk in it:

    # first, filter to only the states that we want
    # good first step as this will elminate unecessary calculations on rows we don't need
    chunk = chunk[chunk["BUYER_STATE"].isin(prescription_states)]


    
    # extract year out of date column

    #chunk['DATE'] = pd.to_datetime(chunk['TRANSACTION_DATE'], format='%m%d%Y')

    # pull out only the year from the date field
    #chunk['year']= chunk['DATE'].dt.year

    #chunk["year"] = chunk["TRANSACTION_DATE"].apply(lambda x: x[-4:])
    chunk["year"] = chunk["TRANSACTION_DATE"].apply(lambda x: int(find_year(x)))

    #chunk = chunk[chunk["year"] > 2012]


    # ensure dtypes for faster calculation below
    float_cols = ["MME_Conversion_Factor", "dos_str", "DOSAGE_UNIT"]
    chunk[float_cols] = chunk[float_cols].astype("float64")



    # calculate MME
    chunk["MME"] = chunk["dos_str"] * chunk["MME_Conversion_Factor"] * chunk["DOSAGE_UNIT"]

    chunk = chunk[cols_to_keep]

    # ensure we're working in the correct date range
    # could map a dict with states + desired years to make this faster later
    #chunk = chunk[chunk["year"] > 2012]
    #filtered_chunk = filtered_chunk[filtered_chunk["year"] < 2016]


    # calculate int cols
    
    int_cols = ["BUYER_ZIP", "year"]
    chunk[int_cols] = chunk[int_cols].astype("int64")

    df_prescriptions = pd.concat([df_prescriptions, chunk])

    print(f"chunk {counter} processed")
    counter+=1
    # if counter == 10:
    #     break


df_prescriptions.drop(columns={"TRANSACTION_DATE"}, inplace=True)
"""

'\n# set FINAL cols we want\ncols_to_keep = ["BUYER_STATE", "BUYER_ZIP", "BUYER_COUNTY", "DRUG_CODE", "DRUG_NAME", "TRANSACTION_DATE", "MME_Conversion_Factor", "dos_str", "DOSAGE_UNIT"]\n\n# set additional columns we need to calculate MME\nchunk_cols = ["dos_str", "DOSAGE_UNIT", "MME_Conversion_Factor"]\n\n\ndf_prescriptions = pd.DataFrame() # empty df - will store data from all txt files\n\n# need 1 row per county month**** could aggergate it here also?\nit = pd.read_csv("00_source_data/arcos_all_washpost.tsv.gz", chunksize=1_000_000, sep=\'\t\', usecols=cols_to_keep, low_memory=False) # may have to change chunksize depending on your computer\'s memory\ncols_to_keep.extend(["year", "MME"])\n\ntemp_df = pd.DataFrame()\ncounter = 0\nfor chunk in it:\n\n    # first, filter to only the states that we want\n    # good first step as this will elminate unecessary calculations on rows we don\'t need\n    chunk = chunk[chunk["BUYER_STATE"].isin(prescription_states)]\n\n\n    \n    # extract ye

## Clean up cause of death data

In [21]:
path = r'00_source_data/cause_of_death' # point to correct folder
filenames = glob.glob(path + "/*.txt") # select all text files in folder

df = pd.DataFrame() # empty df - will store data from all txt files

for f in filenames:
    temp = pd.read_csv(f, index_col=None, header=0, sep='\t')
    # we're getting some extraneous notes at the bottom - let's just drop based on county as these will only be null for these useless notes columns
    temp.dropna(subset={'County'}, inplace=True)
    
    df = pd.concat([df, temp], axis=0, ignore_index=True)

In [22]:
# helper functions to separate county and state

def abtract_state(county):
    """
    Args:
        county (str): county name

    Returns:
        str: state
    """
    return county.split(", ")[1]



def abstract_county(county):
    """
    Args:
        county (str): county name

    Returns:
        str: county
    """
    return county.split(", ")[0]

# apply functions to our df
df["State"] = df.apply(lambda x: abtract_state(x["County"]), axis=1)
df["County"] = df.apply(lambda x: abstract_county(x["County"]), axis=1)

# do not need notes column, let's just drop it here
df.drop(columns={"Notes"}, inplace=True)

df_cause_of_death = df.copy() # keep a copy of this df for later filtering

In [23]:
# now, let's filter our dataframe to be only the states we want
df_cause_of_death = df_cause_of_death[df_cause_of_death["State"].isin(states)]


## Adding in County Population data

[Census county pop. data, 2000-2010](https://www.census.gov/data/tables/time-series/demo/popest/intercensal-2000-2010-counties.html)<br>
[Census county pop. data, 2010-2019](https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html)<br>
For both, just select the appropriate states on the webpage. We will clean and merge as needed in this notebook.



#### Guide to cleaning - 2000s data

The way the 2000s excel files are formatted, we can clean the data in the following way

- load in with header=3
- drop null on any of the populations
    - notes at the bottom will be removed
- drop unnamed 1, 12, and 13
    - these contain redundant data about populations from specific dates
    - Unnamed 12 is 2010s pop - will be redundant as our next dataset has this as well. Using the newer data
- drop first row
    - state as a whole
- rename Unnamed: 0 to county


In [24]:
# init emmpty df for our population data
pops00 = pd.DataFrame()

# end goal - add every excel file in 00_source_data/county_pop/2000s to pops00

path = r"00_source_data/county_pop/2000s/" # point to correct folder
filenames = glob.glob(path + "*.xls")

for f in filenames:

    # read in current file with header = 3
    temp = pd.read_excel(f, header = 3)

    # regex to pull out state from filename
    r = re.search("(2000s)(.)(\w+)", f)[3]
    temp["State"] = r[:2].upper()
    
    # drop null on any of the years
    temp.dropna(subset=[2000], inplace=True)

    #drop useless columns
    temp.drop(columns={"Unnamed: 1", "Unnamed: 12", "Unnamed: 13"}, inplace=True)

    # drop first row
    temp = temp.iloc[1:, :]

    # rename some cols
    temp.rename(columns={"Unnamed: 0": "County"}, inplace=True)

    # remove period at beginning of each county
    temp["County"] = temp["County"].apply(lambda x: x[1:])

    pops00 = pd.concat([pops00, temp], axis=0, ignore_index=True)

# quick peek at the data
pops00.head()


,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,State
0,Adams County,350888.0,359816.0,370753.0,377464.0,384809.0,395146.0,406575.0,415746.0,424913.0,435700.0,CO
1,Alamosa County,14954.0,14956.0,15114.0,15067.0,15217.0,15236.0,15196.0,15180.0,15300.0,15289.0,CO
2,Arapahoe County,491482.0,502393.0,508936.0,513690.0,518971.0,524466.0,531619.0,542039.0,552461.0,563161.0,CO
3,Archuleta County,10020.0,10454.0,10885.0,11089.0,11266.0,11496.0,11937.0,12262.0,12250.0,12169.0,CO
4,Baca County,4501.0,4471.0,4336.0,4117.0,4064.0,3997.0,3933.0,3866.0,3806.0,3767.0,CO


#### Guide to cleaning - 2010s data

The way the 2010s excel files are formatted, we can clean the data in the following way

- load in with header=3
- drop null on any of the populations
    - notes at the bottom will be removed
- drop census, estimates base
- drop first row
    - state as a whole
- rename Unnamed: 0 to county


In [25]:
pops10 = pd.DataFrame()

# add every excel file in 00_source_data/county_pop/2000s to pops00

path = r"00_source_data/county_pop/2010s" # point to correct folder
filenames = glob.glob(path + "/*.xlsx")

for f in filenames:

    # read in current file with header = 3
    temp = pd.read_excel(f, header = 3)

    # regex to pull out state from filename
    r = re.search("(2010s)(.)(\w+)", f)[3]
    temp["State"] = r[:2].upper()
    
    # drop null on any of the years
    temp.dropna(subset=[2010], inplace=True)

    #drop useless columns
    temp.drop(columns={"Census", "Estimates Base"}, inplace=True)

    # drop first row
    temp = temp.iloc[1:, :]

    # rename some cols
    temp.rename(columns={"Unnamed: 0": "County"}, inplace=True)

    # remove period at beginning of each county
    temp["County"] = temp["County"].apply(lambda x: x[1:])

    # strip state from county
    temp["County"] = temp["County"].apply(lambda x: x.split(", ")[0])

    pops10 = pd.concat([pops10, temp], axis=0, ignore_index=True)

# quick peek at the data
pops10.head()


,County,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,State
0,Adams County,443691.0,452201.0,460558.0,469978.0,479946.0,490443.0,497734.0,503590.0,511354.0,517421.0,CO
1,Alamosa County,15515.0,15709.0,15680.0,15787.0,15803.0,15894.0,16053.0,16108.0,16248.0,16233.0,CO
2,Arapahoe County,574747.0,585968.0,596500.0,608467.0,619034.0,630984.0,638950.0,644478.0,651797.0,656590.0,CO
3,Archuleta County,12046.0,12021.0,12132.0,12216.0,12231.0,12387.0,12825.0,13295.0,13730.0,14029.0,CO
4,Baca County,3807.0,3778.0,3722.0,3656.0,3587.0,3555.0,3530.0,3554.0,3584.0,3581.0,CO


In [26]:
# melt both dfs to get tidy format
pops00 = pops00.melt(["County", "State"])
pops10 = pops10.melt(["County", "State"])

# rename columns accordingly
pops00.rename(columns={"variable": "Year", "value": "Population"}, inplace=True)
pops10.rename(columns={"variable": "Year", "value": "Population"}, inplace=True)

# concatenate the two dfs to get all our population data in one place
pops = pd.concat([pops00, pops10], ignore_index=True)


In [27]:
# check that we have the same number of counties between datasets
assert len(pops00["County"].unique()) == len(pops10["County"].unique())

# check that we have the same number of counties every year
# first, create a df with the number of counties per year
pops_county_check = pops.groupby(["State", "Year"])["County"].count().reset_index()

In [28]:
# group the sum of counties by year and state - will help us check if number of counties changes over the years
grouped_states = pops_county_check.groupby(["Year", "State"])["County"].sum().reset_index().rename(columns={"County": "county_count"})

# here's what this looks like
# we get a dataframe of states and years, with the number of counties in each state in each year
grouped_states.head()

,Year,State,county_count
0,2000,CO,64
1,2000,FL,67
2,2000,IL,102
3,2000,MA,14
4,2000,MD,24


In [29]:
# using the above query, we should be able to assert that the number of counties per year is the same
# below statement should always equal zero

assert (grouped_states.duplicated(subset=["Year", "State"]).sum() == 0)
#assert (grouped_states10.duplicated(subset=["Year", "State"]).sum() == 0)


# ensure no duplicate values
assert pops.duplicated().sum() == 0

# loop to check that every state has the same number of counties every year
for state in states:
    assert (pops[pops["State"] == state].Year.value_counts().nunique() == 1), f"error on {state}"

## trying to integrate fip numbers for a better merge

In [30]:
# load in fips data from external source
fips = pd.read_csv("https://github.com/ChuckConnell/articles/raw/master/fips2county.tsv", sep="\t")

In [31]:
# function to get key from value in our abbreviation dictionary
# will help us have consistent formatting across dataframes for merging purposes
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]


keys = get_keys_from_value(abbrev_to_us_state, 'Alabama')
keys # quick peek to make sure it worked

['AL']

In [32]:
# apply the above to entire fips dataframe
fips["state_abbrev"] = fips["StateName"].apply(lambda x: get_keys_from_value(abbrev_to_us_state, x)[0])

# filter fips to appropriate states, now that it's in the correct format
fips = fips[fips["state_abbrev"].isin(states)]

#### Further cleaning of values before merge

In [33]:
# helper function to get rid of the word county in pop df
def remove_county(x):

    if "County" in x:
        return x[:-7]
    else:
        return x


pops["county_test"] = pops["County"].apply(lambda x: remove_county(x))


# fix dona ana and la salle parish
pops["county_test"] = pops["county_test"].apply(lambda x: x.replace("Doña Ana", "Dona Ana"))
fips["CountyName"] = fips["CountyName"].apply(lambda x: x.replace("DoÃ±a Ana", "Dona Ana"))


#pops["county_test"] = pops["county_test"].apply(lambda x: x.replace("La Salle Parish", "La Salle"))


# rename county_test where state is texas and county is la salle to La Salle (TX)
pops.loc[(pops["State"] == "TX") & (pops["county_test"] == "La Salle"), "county_test"] = "La Salle County"
        

In [34]:
# change La Salle county name in fips to La Salle County
fips.loc[fips["CountyName"] == "La Salle", "CountyName"] = "La Salle County"
fips.loc[fips["CountyName"] == "LaSalle Parish", "CountyName"] = "La Salle Parish"
pops.loc[pops["county_test"] == "LaSalle Parish", "county_test"] = "La Salle Parish"



In [35]:
# final merge for population dataset & fip number dataset
pops_copy = pops.merge(fips[["state_abbrev", "CountyFIPS", "StateFIPS", "CountyName"]], left_on=["county_test", "State"], right_on=["CountyName", "state_abbrev"], how="outer", indicator=True)

In [36]:
# should never end up with anything left out of merge
assert len(pops_copy[pops_copy["_merge"] != "both"]) == 0

In [37]:
# add fip numbers to df_prescriptions

# create copies of both dfs so we have a checkpoint to access our old dfs
prescriptions_copy = df_prescriptions.copy()
fips_copy = fips.copy()

In [38]:
# TODO: fix nulls in prescriptions at this point

prescriptions_copy.dropna(subset=["BUYER_COUNTY"],inplace=True)

In [39]:
# make buyer_county all lowercase
prescriptions_copy["BUYER_COUNTY"] = prescriptions_copy["BUYER_COUNTY"].apply(lambda x: x.lower())

# do the same for fips
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: x.lower())

In [40]:
# remove county and parish from fips_copy

def remove_parish(x):

    if "parish" in x:
        return x[:-7]
    else:
        return x


# prescription dataset has similar format - match fips to this format
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: remove_county(x))
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: remove_parish(x))

def expand_saint(x):

    if "st." in x:
        return x.replace("st.", "saint")
    else:
        return x

# fix various other inconsistencies
# left only values first
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: expand_saint(x))

fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: x.replace("desoto", "de soto"))
prescriptions_copy["BUYER_COUNTY"] = prescriptions_copy["BUYER_COUNTY"].apply(lambda x: x.replace("desoto", "de soto"))
prescriptions_copy["BUYER_COUNTY"] = prescriptions_copy["BUYER_COUNTY"].apply(lambda x: x.replace("st john the baptist", "saint john the baptist"))

fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: x.replace("dekalb", "de kalb"))
prescriptions_copy["BUYER_COUNTY"] = prescriptions_copy["BUYER_COUNTY"].apply(lambda x: x.replace("dekalb", "de kalb"))

# fix right only values

prescriptions_copy["BUYER_COUNTY"] = prescriptions_copy["BUYER_COUNTY"].apply(lambda x: x.replace("desoto", "de soto"))



# function to remove apostrophes from county names
def remove_apostrophe(x):
    
    if "'" in x:
        return x.replace("'", "")
    else:
        return x


# apply to fips
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: remove_apostrophe(x))

# replace lasalle with la salle in fips copy
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: x.replace("lasalle", "la salle"))

# replace dewitt with de witt in prescriptions copy
prescriptions_copy["BUYER_COUNTY"] = prescriptions_copy["BUYER_COUNTY"].apply(lambda x: x.replace("dewitt", "de witt"))


In [113]:
prescriptions_fips = prescriptions_copy.merge(fips_copy, left_on=["BUYER_COUNTY", "BUYER_STATE"], right_on=["CountyName", "state_abbrev"], how="outer", indicator=True)

# capitalize year and month columns
prescriptions_fips.rename(columns={"year": "Year", "month": "Month"}, inplace=True)

#### Filling missing prescription shipment values

Since we have plenty of values joined with right_only indicator status, we know that some counties in our FIPS dataset is not merging correctly to our prescriptions dataset. There are 376 rows where this occurs. After extensive data cleaning and checking of counties, we believe these values should be filled in with zero, as we can assume no prescriptions were shipped to these counties in their given years.


## Analyzing the problem

We have to main issues to overcome in terms of missing values

1) We have 376 values not joining back to our main dataframe from the FIPS dataframe
2) A handful of counties have shipments in some years, and no shipments in other years


The solution to 2) will be touched on later in this notebook. As for 1), the solution is as follows:

### Solution:

We are assuming that the right_only joins (counties present in FIPS data but not prescription shipment data) implies that there were zero shipments for a given county throughout our entire range of years. Therefore, we cannot drop these values as we'd be ignoring counties that do not have any shipments and, therefore, would result in a biased analysis. To solve this issue, we will perform the following:

1) create lists for various fields in our dataframe where we see this right_only join
2) expand these lists out by a factor of 9 (because we are studying 9 years of interest)
    - this is done through a list comprehension that multiplies each value by a range() object, therefore expanding our lists
3) adding a vector of zeros for the MME column
4) concatenating this data to our prescriptions dataframe

In [114]:
# quick look at some of the right only merges
prescriptions_fips[prescriptions_fips["_merge"] != "both"]

,BUYER_STATE,BUYER_COUNTY,Year,MME,StateFIPS,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY,state_abbrev,_merge
3918,NaN,NaN,NaN,NaN,8,23,costilla,Colorado,8023,CO,CO | COSTILLA,CO,right_only
3919,NaN,NaN,NaN,NaN,8,33,dolores,Colorado,8033,CO,CO | DOLORES,CO,right_only
3920,NaN,NaN,NaN,NaN,8,57,jackson,Colorado,8057,CO,CO | JACKSON,CO,right_only
3921,NaN,NaN,NaN,NaN,17,1,adams,Illinois,17001,IL,IL | ADAMS,IL,right_only
3922,NaN,NaN,NaN,NaN,17,3,alexander,Illinois,17003,IL,IL | ALEXANDER,IL,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4289,NaN,NaN,NaN,NaN,48,499,wood,Texas,48499,TX,TX | WOOD,TX,right_only
4290,NaN,NaN,NaN,NaN,48,501,yoakum,Texas,48501,TX,TX | YOAKUM,TX,right_only
4291,NaN,NaN,NaN,NaN,48,503,young,Texas,48503,TX,TX | YOUNG,TX,right_only
4292,NaN,NaN,NaN,NaN,48,505,zapata,Texas,48505,TX,TX | ZAPATA,TX,right_only


In [43]:
# df of all the nulls as stated above
# can take year's null values because this column will have not joined back to the records we're interested in
nans = prescriptions_fips[prescriptions_fips["Year"].isnull()]

# create lists that pull columns from each of these non-joined records
missing_counties = nans.CountyName
missing_states=nans.StateAbbr
missing_fips = nans.CountyFIPS
missing_state_fips = nans.StateFIPS

In [44]:
# expand each list out so there are 9 years for each record
missing_counties = [item for item in missing_counties for i in range(9)]
missing_states = [item for item in missing_states for i in range(9)]
missing_fips = [item for item in missing_fips for i in range(9)]
missing_state_fips = [item for item in missing_state_fips for i in range(9)]

In [120]:
assert len(missing_counties) == len(missing_states) == len(missing_fips) == len(missing_state_fips)

In [121]:
# create an iterable list of years
# will be cycled through and added to a new list
# this list should match one of each year (2006-2014) for each of our records in our lists above
years = list(np.arange(2006, 2015, 1))

print(f"length of each of our lists: {len(missing_counties)}")

# list to be appended to our df
years_for_df = []

# 3384, as seen in print above, is the number of records we have in our lists
# we want to divide by the number of years we want for each record, so we divide by 9
# this results in 374 records for each year
for year in range(int(3384/9)):
    for subyear in years:
        years_for_df.append(subyear)

length of each of our lists: 3384


In [47]:
# create first missing df, append values derived above
missing_df = pd.DataFrame()

missing_df["BUYER_COUNTY"] = missing_counties
missing_df["BUYER_STATE"] = missing_states
missing_df["CountyFIPS"] = missing_fips
missing_df["StateFIPS"] = missing_state_fips
missing_df["Year"] = years_for_df

In [124]:
assert len(missing_df.Year.value_counts()) == 9, "double check that we have 9 years for each record"

#### Fixing our second issue

Now, we want to tackle the problem where some counties have shipments in some years, but not others. As can be seen below, we are missing a few counties in each year. The good news on this front is that there shouldn't be very many records to fix, so we likely do not have to be robust to the possibility of duplicate county names across states (more concrete proof of this is further down in the notebook).

In [125]:
prescriptions_fips.Year.value_counts()

2006.0    437
2007.0    436
2010.0    436
2009.0    435
2011.0    435
2012.0    435
2013.0    435
2014.0    435
2008.0    434
Name: Year, dtype: int64

### Solving a few missing counties in specific years

First, we'll make a list of every county below. We'll copy this list for every year in which we want data (2006-2014). Then, we'll check our prescription dataframe for records in which these counties **do not appear with an associated year**. Given that there are not too many missing values (as per value counts in cell above), we can manually check each value to make sure it is actually missing in our dataframe (and therefore, check that our logic is correct).

In [64]:
# make list of unique counties
counties = prescriptions_fips.BUYER_COUNTY.unique()

# duplicate this list for every year we want
counties2006 = counties.copy()
counties2007 = counties.copy()
counties2008 = counties.copy()
counties2009 = counties.copy()
counties2010 = counties.copy()
counties2011 = counties.copy()
counties2012 = counties.copy()
counties2013 = counties.copy()
counties2014 = counties.copy()


In [71]:
# make a missing list for every year we want
# iterate through each county in the list and check if it is in the dataframe for that year
# admittedly, this is a suboptimal solution, but given the small amount of missing records it is workable

missing06 = []
for county in counties2006:
    if county not in prescriptions_fips[(prescriptions_fips["Year"] == 2006)]["BUYER_COUNTY"].unique():
        missing06.append(county)

missing07 = []
for county in counties2007:
    if county not in prescriptions_fips[(prescriptions_fips["Year"] == 2007)]["BUYER_COUNTY"].unique():
        missing07.append(county)

missing08 = []
for county in counties2008:
    if county not in prescriptions_fips[(prescriptions_fips["Year"] == 2008)]["BUYER_COUNTY"].unique():
        missing08.append(county)

missing09 = []
for county in counties2009:
    if county not in prescriptions_fips[(prescriptions_fips["Year"] == 2009)]["BUYER_COUNTY"].unique():
        missing09.append(county)

missing10 = []
for county in counties2010:
    if county not in prescriptions_fips[(prescriptions_fips["Year"] == 2010)]["BUYER_COUNTY"].unique():
        missing10.append(county)

missing11 = []
for county in counties2011:
    if county not in prescriptions_fips[(prescriptions_fips["Year"] == 2011)]["BUYER_COUNTY"].unique():
        missing11.append(county)

missing12 = []
for county in counties2012:
    if county not in prescriptions_fips[(prescriptions_fips["Year"] == 2012)]["BUYER_COUNTY"].unique():
        missing12.append(county)

missing13 = []
for county in counties2013:
    if county not in prescriptions_fips[(prescriptions_fips["Year"] == 2013)]["BUYER_COUNTY"].unique():
        missing13.append(county)

missing14 = []
for county in counties2014:
    if county not in prescriptions_fips[(prescriptions_fips["Year"] == 2014)]["BUYER_COUNTY"].unique():
        missing14.append(county)

### double check that there are no duplicate counties across states

In [87]:
# combine all missing lists into one
missing_total = missing06 + missing07 + missing08 + missing09 + missing10 + missing11 + missing12 + missing13 + missing14

# check county values against this list
prescriptions_fips[prescriptions_fips["BUYER_COUNTY"].isin(set(missing_total))]

,BUYER_STATE,BUYER_COUNTY,Year,MME,StateFIPS,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY,state_abbrev,_merge
117,CO,custer,2006.0,2.724300,8,27,custer,Colorado,8027,CO,CO | CUSTER,CO,both
118,CO,custer,2007.0,3.027000,8,27,custer,Colorado,8027,CO,CO | CUSTER,CO,both
119,CO,custer,2009.0,0.605400,8,27,custer,Colorado,8027,CO,CO | CUSTER,CO,both
120,CO,custer,2010.0,1.513500,8,27,custer,Colorado,8027,CO,CO | CUSTER,CO,both
121,CO,custer,2011.0,3.027000,8,27,custer,Colorado,8027,CO,CO | CUSTER,CO,both
122,CO,custer,2012.0,3.329700,8,27,custer,Colorado,8027,CO,CO | CUSTER,CO,both
123,CO,custer,2013.0,5.751300,8,27,custer,Colorado,8027,CO,CO | CUSTER,CO,both
124,CO,custer,2014.0,4.017900,8,27,custer,Colorado,8027,CO,CO | CUSTER,CO,both
332,CO,mineral,2010.0,0.302700,8,79,mineral,Colorado,8079,CO,CO | MINERAL,CO,both
706,FL,glades,2009.0,2627.311200,12,43,glades,Florida,12043,FL,FL | GLADES,FL,both


The good news is that **none of our missing counties have duplicate names across states**, so we can be sure our method for deriving these values is workable. If there was a Glades County in both Florida and Colorado, for example, the above method would be too ambiguous and we'd need to save the state variable in our missing lists as well.

In [93]:
# make list of years the same dimension as missing total that has the appropriate year based on length of missing total
# in simpler terms, this is a list of 2006s, 2007s, etc. that is the same length of its corresponding missing list
missing_yrs06 = [2006 for item in missing06]
missing_yrs07 = [2007 for item in missing07]
missing_yrs08 = [2008 for item in missing08]
missing_yrs09 = [2009 for item in missing09]
missing_yrs10 = [2010 for item in missing10]
missing_yrs11 = [2011 for item in missing11]
missing_yrs12 = [2012 for item in missing12]
missing_yrs13 = [2013 for item in missing13]
missing_yrs14 = [2014 for item in missing14]

# concat into one list
missing_yrs = missing_yrs06 + missing_yrs07 + missing_yrs08 + missing_yrs09 + missing_yrs10 + missing_yrs11 + missing_yrs12 + missing_yrs13 + missing_yrs14

In [95]:
assert len(missing_total) == len(missing_yrs)

In [107]:
# create second missing df
smaller_missing_df = pd.DataFrame()

# add two columns calculated from lists above
smaller_missing_df["BUYER_COUNTY"] = missing_total
smaller_missing_df["Year"] = missing_yrs

# save state of df for checking after merge
smaller_df_check = smaller_missing_df.copy()

# since we know there are no duplicate counties for these values, we will left join the state back to this df with confidence that the result is sound
smaller_missing_df = pd.merge(smaller_missing_df, prescriptions_fips[["BUYER_COUNTY", "StateFIPS", "CountyFIPS", "BUYER_STATE"]], how="left", left_on="BUYER_COUNTY", right_on="BUYER_COUNTY")

# drop duplicates to get rid of extra columns that appear as a result of join
smaller_missing_df.drop_duplicates(subset={"BUYER_COUNTY", "Year"}, inplace=True)

In [109]:
assert len(smaller_missing_df) == len(smaller_df_check)

### Joining these all back together

First, we need to clean the datasets to make sure we have only the columns we need so they concatenate properly.

In [130]:
# subset prescription fips to only what we need
prescriptions_fips = prescriptions_fips[["BUYER_STATE", "BUYER_COUNTY", "CountyFIPS", "StateFIPS", "Year", "MME"]]

In [131]:
# reorganize missing_df to match prescriptions_fips
missing_df = missing_df[["BUYER_STATE", "BUYER_COUNTY", "CountyFIPS", "StateFIPS", "Year"]]
missing_df["MME"] = 0

In [132]:
# reorganize smaller missing df to match prescriptions_fips
smaller_missing_df = smaller_missing_df[["BUYER_STATE", "BUYER_COUNTY", "CountyFIPS", "StateFIPS", "Year"]]
smaller_missing_df["MME"] = 0

concatenate all three dataframes

In [149]:
prescriptions = pd.concat([prescriptions_fips, missing_df, smaller_missing_df], axis=0, ignore_index=True)

In [150]:
# checking how one of the values looks now
prescriptions[prescriptions["BUYER_COUNTY"] == "mineral"].sort_values("Year")

,BUYER_STATE,BUYER_COUNTY,CountyFIPS,StateFIPS,Year,MME
7678,CO,mineral,8079,8,2006.0,0.0000
7680,CO,mineral,8079,8,2007.0,0.0000
7683,CO,mineral,8079,8,2008.0,0.0000
7685,CO,mineral,8079,8,2009.0,0.0000
332,CO,mineral,8079,8,2010.0,0.3027
7686,CO,mineral,8079,8,2011.0,0.0000
7687,CO,mineral,8079,8,2012.0,0.0000
7688,CO,mineral,8079,8,2013.0,0.0000
7689,CO,mineral,8079,8,2014.0,0.0000


In [151]:
# drop leftover duplicates, if any
prescriptions.drop_duplicates(subset={"BUYER_STATE","BUYER_COUNTY", "Year"}, keep="first", inplace=True)

### adding fips to our cause of death data

In [155]:
# create copies of both dfs

cause_of_death_copy = df_cause_of_death.copy()
fips_copy = fips.copy()

In [156]:
# remove county once again
cause_of_death_copy["County"] = cause_of_death_copy["County"].apply(lambda x: remove_county(x))


# clean some other miscellaneous values up

cause_of_death_copy["County"] = cause_of_death_copy["County"].apply(lambda x: x.replace("LaSalle Parish", "La Salle Parish"))
cause_of_death_copy["County"] = cause_of_death_copy["County"].apply(lambda x: x.replace("DeBaca", "De Baca"))
cause_of_death_copy["County"] = cause_of_death_copy["County"].apply(lambda x: x.replace("La Salle", "La Salle County"))
cause_of_death_copy["County"] = cause_of_death_copy["County"].apply(lambda x: x.replace("La Salle County Parish", "La Salle Parish"))



# expand mckean to mc kean in fips_copy
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: x.replace("McKean", "Mc Kean"))


In [157]:
cause_of_death_fips = cause_of_death_copy.merge(fips_copy, left_on=["County", "State"], right_on=["CountyName", "state_abbrev"], how="outer", indicator=True)

#### Not all counties joining back to cause of death dataset

If the number of people in a given category (eg. one county/year/cause of death category) is less than 10, those records do not appear in this data. There is also a technicality in the number of total deaths vs. drug deaths (which we are interested in).

The example we are given is that if a county has 20 deaths unrelated to drugs and alcohol, and only 7 related to alcohol, only the former figure will be reported. In the next notebook (pick_states.ipynb), we will filter by cause of death. In this notebook, since we still have all causes of death, we will impute for every missing value.

To impute this data, we will fill in missing values with **a random integer from 0 to 9**. We thought of drawing from a normal distribution, but this implies negative values could be attained. We could take their absolute values to negate this effect, but then we are no longer drawing from a *true* normal distribution, so we chose to pick random values in our range.

In [158]:
# function to replace null value with a random integer from 0 to 10 with a normal distribution
def value_imputer(x):
    if pd.isnull(x):
        return random.randint(0, 9)
    else:
        return x

cause_of_death_fips["Deaths"] = cause_of_death_fips["Deaths"].apply(lambda x: value_imputer(x))

In [159]:
# quick look at our new imputed data
cause_of_death_fips[cause_of_death_fips["_merge"] != "both"].Deaths.value_counts()

4    2
0    1
8    1
7    1
9    1
Name: Deaths, dtype: int64

### Adding Population to final DataFrames

For pop_fips, cause_of_death_fips, and prescription_fips. Steps needed:

- Create unique ID from county FIPS and state FIPS
- Merge population dataset based on this

In [ ]:
cause_of_death_fips = cause_of_death_fips[cause_of_death_fips["_merge"] == "both"]
#pops_copy = pops_copy[cause_of_death_fips["_merge"] == "both"]
#prescriptions_fips = prescriptions_fips[prescriptions_fips["_merge"] == "both"]


# drop merge columns
cause_of_death_fips.drop(columns=["_merge"], inplace=True)
prescriptions_fips.drop(columns=["_merge"], inplace=True)
pops_copy.drop(columns=["_merge",], inplace=True)

In [ ]:
# create unique FIP from county and state fips

cause_of_death_fips["FIP_unique"] = cause_of_death_fips["CountyFIPS"].apply(lambda x: str(x)) + cause_of_death_fips["StateFIPS"].apply(lambda x: str(x))
prescriptions_fips["FIP_unique"] = prescriptions_fips["CountyFIPS"].apply(lambda x: str(x)) + prescriptions_fips["StateFIPS"].apply(lambda x: str(x))
pops_copy["FIP_unique"] = pops_copy["CountyFIPS"].apply(lambda x: str(x)) + pops_copy["StateFIPS"].apply(lambda x: str(x))


In [ ]:
# TODO: add some sort of assert here. not sure what it should be yet


In [ ]:
# create final prescriptions dataset with populations
# can safely left join here, because we only need records in the prescriptions dataset
prescriptions = prescriptions_fips.merge(pops_copy, on=["FIP_unique", "Year"], how="left", indicator=True)

#assert (prescriptions["_merge"] == "both").all()

In [ ]:
# STILL HAVE WASHINGTON YEARS HERE
prescriptions[prescriptions["BUYER_STATE"] == "WA"].Year.value_counts()

2006.0    327
2007.0    325
2008.0    324
2009.0    321
2010.0    317
2011.0    315
2012.0    311
2013.0    309
2014.0    305
Name: Year, dtype: int64

In [ ]:
# one more assert to check length
assert len(prescriptions) == len(prescriptions_fips)


In [ ]:
# drop some useless columns
prescriptions.drop(columns=["_merge", "CountyName_y", "StateFIPS_y", "CountyFIPS_y","state_abbrev_y", "County", "CountyFIPS_3"], inplace=True)

# rename x columns
prescriptions.rename(columns={"CountyName_x": "CountyName", "StateFIPS_x": "StateFIPS", "CountyFIPS_x": "CountyFIPS", "state_abbrev_x": "state_abbrev"}, inplace=True)


In [ ]:
# create final cause of death dataset with populations
# can safely left join here, because we only need records in the cause of death dataset
cause_of_death = cause_of_death_fips.merge(pops_copy, on=["FIP_unique", "Year"], how="left", indicator=True)

assert cause_of_death_fips.Deaths.isnull().sum() == 0

In [ ]:
# drop some useless columns
cause_of_death.drop(columns=["_merge", "CountyName_y", "StateFIPS_y", "CountyFIPS_y","state_abbrev_y", "County_y", "CountyFIPS_3", "State_y"], inplace=True)

# rename x columns
cause_of_death.rename(columns={"County_x": "County", "Year_x": "Year", "State_x": "State", "StateFIPS_x": "StateFIPS", "CountyFIPS_x": "CountyFIPS", "state_abbrev_x": "state_abbrev", "CountyName_x": "CountyName"}, inplace=True)


In [ ]:
# asserts to make sure we didn't lose any records from our original datasets

assert len(df_cause_of_death) == len(cause_of_death)
assert len(df_prescriptions) == len(prescriptions)

AssertionError: 

#### Export main, unjoined datasets in case we need them

In [ ]:
cause_of_death.to_csv("20_intermediate_files/cause_of_death_clean.csv", index=False)
prescriptions.to_csv("20_intermediate_files/arcos_all_washpost_clean.csv", index=False)

KeyboardInterrupt: 

## Final 3 datasets

We should have: (UNSURE IF WE SHOULD EXTEND DATE RANGES, CURRENTLY 3 YEARS BEFORE AND AFTER POLICY IMPLEMENTATION)

- Florida and Georgia 2007 - 2013
- Texas and Oklahoma 2004 - 2010
- Washington and Oregon 2009 - 2015

### Drug overdose - broken down by state

In [ ]:
# Florida and Georgia

prescriptions_fl = prescriptions.copy()
prescriptions_wa = prescriptions.copy()

prescriptions_fl = prescriptions_fl[(prescriptions_fl["BUYER_STATE"] == "FL") | (prescriptions_fl["BUYER_STATE"].isin(fl_states))]
prescriptions_wa = prescriptions_wa[(prescriptions_wa["BUYER_STATE"] == "WA") | (prescriptions_wa["BUYER_STATE"]).isin(wa_states)]



# filter appropriate years
fl_start = 2007
fl_end = 2013

# tx will only be used for overdose deaths
tx_start = 2004
tx_end = 2010

wa_start = 2009
wa_end = 2015


prescriptions_fl = prescriptions_fl[(prescriptions_fl["Year"] >= fl_start) & (prescriptions_fl["Year"] <= fl_end)]
prescriptions_wa = prescriptions_wa[(prescriptions_wa["Year"] >= wa_start) & (prescriptions_wa["Year"] <= wa_end)]


### Cause of death - broken down by state

In [ ]:
deaths_fl = cause_of_death.copy()
deaths_tx = cause_of_death.copy()
deaths_wa = cause_of_death.copy()

deaths_fl = deaths_fl[(deaths_fl["StateName"] == "Florida") | (deaths_fl["State"].isin(fl_states))]
deaths_tx = deaths_tx[(deaths_tx["StateName"] == "Texas") | (deaths_tx["State"].isin(tx_states))]
deaths_wa = deaths_wa[(deaths_wa["StateName"] == "Washington") | (deaths_wa["State"].isin(wa_states))]

deaths_fl = deaths_fl[(deaths_fl["Year"] >= fl_start) & (deaths_fl["Year"] <= fl_end)]
deaths_tx = deaths_tx[(deaths_tx["Year"] >= tx_start) & (deaths_tx["Year"] <= tx_end)]  
deaths_wa = deaths_wa[(deaths_wa["Year"] >= wa_start) & (deaths_wa["Year"] <= wa_end)]

In [ ]:
prescriptions_wa.Year.value_counts()

2013.0    2681681
2011.0    2656850
2012.0    2654508
2010.0    2562310
2014.0    2433600
2009.0    2419344
Name: Year, dtype: int64

In [ ]:
# final assert to check years

assert prescriptions_fl.Year.unique().tolist() == list(range(fl_start, fl_end + 1))
assert prescriptions_wa.Year.unique().tolist() == list(range(wa_start, wa_end + 1))

assert deaths_fl.Year.unique().tolist() == list(range(fl_start, fl_end + 1))
assert deaths_tx.Year.unique().tolist() == list(range(tx_start, tx_end + 1))
assert deaths_wa.Year.unique().tolist() == list(range(wa_start, wa_end + 1))

AssertionError: 

### Export all to csv

In [ ]:
#prescriptions_fl.to_csv("20_intermediate_files/prescriptions_fl.csv", index=False)
#prescriptions_wa.to_csv("20_intermediate_files/prescriptions_wa.csv", index=False)

deaths_fl.to_csv("20_intermediate_files/deaths_fl.csv", index=False)
deaths_tx.to_csv("20_intermediate_files/deaths_tx.csv", index=False)
deaths_wa.to_csv("20_intermediate_files/deaths_wa.csv", index=False)

### Save all as parquet files

In [ ]:
prescriptions_fl.to_parquet('20_intermediate_files/prescriptions_fl.parquet', engine='fastparquet', row_group_offsets=10_000_000)
prescriptions_wa.to_parquet('20_intermediate_files/prescriptions_wa.parquet', engine='fastparquet', row_group_offsets=10_000_000)

## Notes for the group

- may need to filter out a couple more columns - haven't done this yet as I don't want to accidentally delete something we need
- overdose data is only broken down by year unless i messed something up - overdose analysis will have to be less granular